<a href="https://colab.research.google.com/github/ashlesh003/QA-DOCS/blob/main/AI%20Model/QA_model_approach%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 72.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
# Load a pre-trained model explicitly
qa_pipeline = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:

# Function to load and preprocess the text file
# def load_text_file(file_path):
#     with open(file_path, 'r') as file:
#         return file.read()

import fitz
#Extract Text from PDF
def load_text_file(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
# Function to split text into smaller chunks (sentences or paragraphs)
def split_text_into_chunks(text, chunk_size=10):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


In [ ]:
# Function to rank the chunks based on relevance to the question
def rank_chunks_by_relevance(question, chunks):
    vectorizer = TfidfVectorizer().fit_transform([question] + chunks)
    vectors = vectorizer.toarray()
    question_vector = vectors[0]
    chunk_vectors = vectors[1:]

    # Calculate cosine similarity between the question and each chunk
    similarity_scores = np.dot(chunk_vectors, question_vector) / (
        np.linalg.norm(chunk_vectors, axis=1) * np.linalg.norm(question_vector) + 1e-10  # Small epsilon to avoid division by zero
    )

    # Sort chunks by similarity score (higher is more relevant)
    ranked_chunks = sorted(zip(similarity_scores, chunks), reverse=True)
    return ranked_chunks  # Return both score and chunk

In [ ]:
# Function to generate an answer based on the most relevant chunks with a threshold
def get_detailed_answer(question, chunks, top_n=5, relevance_threshold=0.01):
    ranked_chunks = rank_chunks_by_relevance(question, chunks)

    # Check if the highest similarity score is below the relevance threshold
    if ranked_chunks[0][0] < relevance_threshold:
        return "No result found"

    # Combine the top N relevant chunks to expand the context
    combined_context = " ".join([chunk for score, chunk in ranked_chunks[:top_n]])

    # Use the QA pipeline to generate the answer
    result = qa_pipeline(question=question, context=combined_context)
    return result['answer']

In [ ]:

# Load and process the text file
file_path = 'SolarSystem.pdf'
text = load_text_file(file_path)

# Split the text into chunks for better analysis
chunks = split_text_into_chunks(text)

In [ ]:
# Take user input for the question
question = input("Please enter your question: ")

# Get the detailed answer based on the top relevant chunks
answer = get_detailed_answer(question, chunks, top_n=5, relevance_threshold=0.01)  # Adjust top_n and threshold as needed

# Save the answer to ans.txt
with open('ans.txt', 'w') as answer_file:
    answer_file.write(f"Question: {question}\n")
    answer_file.write(f"Answer: {answer}\n")

print("The answer has been saved to ans.txt.")


Please enter your question: Jupiter has how many moons
The answer has been saved to ans.txt.


# Another Method

In [ ]:
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
# Load a pre-trained model explicitly
qa_pipeline = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")

In [ ]:
# Function to load and preprocess the text file
def load_text_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [ ]:
# Function to split text into smaller chunks (sentences or paragraphs)
def split_text_into_chunks(text, chunk_size=200):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [ ]:
# Function to rank the chunks based on relevance to the question
def rank_chunks_by_relevance(question, chunks):
    vectorizer = TfidfVectorizer().fit_transform([question] + chunks)
    vectors = vectorizer.toarray()
    question_vector = vectors[0]
    chunk_vectors = vectors[1:]

    # Calculate cosine similarity between the question and each chunk
    similarity_scores = np.dot(chunk_vectors, question_vector) / (np.linalg.norm(chunk_vectors, axis=1) * np.linalg.norm(question_vector))

    # Sort chunks by similarity score
    ranked_chunks = sorted(zip(similarity_scores, chunks), reverse=True)
    return [chunk for score, chunk in ranked_chunks]

In [ ]:
# Function to generate an answer based on the most relevant chunk
def get_answer_from_relevant_chunk(question, chunks):
    ranked_chunks = rank_chunks_by_relevance(question, chunks)

    # Select the most relevant chunk
    most_relevant_chunk = ranked_chunks[0]

    # Use the QA pipeline to generate the answer
    result = qa_pipeline(question=question, context=most_relevant_chunk)
    return result['answer']

In [ ]:
# Load and process the text file
file_path = 'info.txt'
text = load_text_file(file_path)

# Split the text into chunks for better analysis
chunks = split_text_into_chunks(text)

In [ ]:
# Take user input for the question
question = input("Please enter your question: ")

# Get the answer based on the most relevant chunk
answer = get_answer_from_relevant_chunk(question, chunks)

# Save the answer to ans.txt
with open('ans.txt', 'w') as answer_file:
    answer_file.write(f"Question: {question}\n")
    answer_file.write(f"Answer: {answer}\n")

print("The answer has been saved to ans.txt.")